# Projet : Système de détection de la fatigue oculaire

## Objectif du projet
Ce projet a pour but de développer un modèle de Machine Learning capable de détecter la fatigue oculaire à partir d’images, afin de prévenir les risques liés à la conduite.

---
## Structure du Notebook

Ce notebook est organisé en plusieurs parties principales :

 1- Collecte des données
 
 2- Prétraitement des données
 
 3- Entraînement du modèle
 
 4- Évaluation du modèle
 
 5- Visualisation et interprétation
 
---

## Collecte des données

Cette section contient le code permettant de récupérer et d’organiser les données utilisées dans ce projet.

Le dataset provient de la plateforme **Kaggle** et a été téléchargé manuellement, puis décompressé localement dans le répertoire de travail du projet.

Chaque sous-partie ci-dessous traite une étape spécifique :
- Vérification de la structure des données
- Renommer les sous-dossiers pour améliorer la lisibilité
- Chargement des images depuis les sous-dossiers

In [3]:
import zipfile
import os

# Chemin du fichier ZIP
zip_path = r'C:\Users\hp\Desktop\jupyter_notebook\driver-drowsiness-dataset-ddd.zip'

# Répertoire de destination
extract_path = r'C:\Users\hp\Desktop\jupyter_notebook'

# Vérifier si le fichier existe avant de l'extraire
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        print("Extraction en cours...")
        zip_ref.extractall(extract_path)
        print("Extraction terminée avec succès !")
else:
    print(f"Le fichier {zip_path} n'existe pas.")


Extraction en cours...
Extraction terminée avec succès !


In [5]:
import os

# Chemin du dataset
dataset_path = r'C:\Users\hp\Desktop\jupyter_notebook\Driver Drowsiness Dataset (DDD)'

# --- Renommer le dossier 'Drowsy' en 'yeux_fermés'
try:
    os.rename(os.path.join(dataset_path, 'Drowsy'), os.path.join(dataset_path, 'yeux_fermés'))
    print("Dossier 'Drowsy' renommé en 'yeux_fermés'")
except FileNotFoundError:
    print("Le dossier 'Drowsy' n'existe pas.")
except Exception as e:
    print(f"Erreur lors du renommage de 'Drowsy' : {e}")

# --- Renommer le dossier 'Non Drowsy' en 'yeux_ouverts'
try:
    os.rename(os.path.join(dataset_path, 'Non Drowsy'), os.path.join(dataset_path, 'yeux_ouverts'))
    print("Dossier 'Non Drowsy' renommé en 'yeux_ouverts'")
except FileNotFoundError:
    print("Le dossier 'Non Drowsy' n'existe pas.")
except Exception as e:
    print(f"Erreur lors du renommage de 'Non Drowsy' : {e}")

# Vérifier le résultat
print("\nContenu du dossier après renommage :")
print(os.listdir(dataset_path))


Le dossier 'Drowsy' n'existe pas.
Le dossier 'Non Drowsy' n'existe pas.

Contenu du dossier après renommage :
['yeux_fermés', 'yeux_ouverts']


In [ ]:
# --- Chargement des images depuis les sous-dossiers :
import os
from PIL import Image

dataset_path = r'C:\Users\hp\Desktop\jupyter_notebook\Driver Drowsiness Dataset (DDD)'

# Liste pour stocker les images
images = []
# Liste pour stocker les étiquettes correspondantes
labels = []

# NB: Chaque sous-dossier correspond à une classe :
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):  # Vérifier si c'est un dossier
        print(f"Chargement des images de la classe : {folder}")
        
        # Parcourir chaque image dans le sous-dossier
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                with Image.open(img_path) as img:
                    # Ajouter l'image et son label (nom du sous-dossier) aux listes
                    images.append(img)
                    labels.append(folder)  # L'étiquette est le nom du sous-dossier
            except Exception as e:
                print(f"Erreur avec l'image {img_path}: {e}")

# Afficher le nombre d'images collectées
print(f"Nombre total d'images collectées : {len(images)}")


Chargement des images de la classe : yeux_fermés


## Prétraitement des données

Dans cette section, nous préparons les données brutes pour notre modèle de Machine Learning afin de garantir que ce dernier reçoive des données propres, cohérentes et exploitables.

Les étapes de prétraitement appliquées sont les suivantes :

### 1- Redimensionnement des images
Toutes les images seront redimensionnées à une taille fixe (par exemple 64x64 pixels) afin d’uniformiser les dimensions et réduire le coût de calcul.

### 2- Normalisation
Les valeurs des pixels seront normalisées (généralement entre 0 et 1) pour faciliter l’apprentissage du modèle et améliorer la convergence.

### 3- Attribution des étiquettes
Chaque image sera associée à une étiquette numérique selon sa classe :
- `0` : yeux fermés
- `1` : yeux ouverts

### 4- Division en jeux d’entraînement et de test
Les données seront divisées en deux ensembles :
- **Entraînement** : pour que le modèle apprenne à reconnaître les classes
- **Test** : pour évaluer la performance du modèle sur des données jamais vues


In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Chemin du dataset
dataset_path = r'C:\Users\hp\Desktop\jupyter_notebook\Driver Drowsiness Dataset (DDD)'

# Dimensions des images
IMG_SIZE = (64, 64)

# Dictionnaire pour mapper les labels
label_map = {
    'yeux_fermés': 0,
    'yeux_ouverts': 1
}

# Listes pour stocker les données et les labels
data = []
labels = []

# Parcourir les dossiers et charger les images
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        print(f'Prétraitement des images de la classe : {folder}')
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                # Ouvrir l'image et la redimensionner
                with Image.open(img_path) as img:
                    img = img.resize(IMG_SIZE)
                    img_array = np.array(img) / 255.0  # Normalisation

                    # Vérifier que l'image est bien en format RGB
                    if img_array.shape == (64, 64, 3):
                        data.append(img_array)
                        labels.append(label_map[folder])
            except Exception as e:
                print(f"Erreur avec l'image {img_path}: {e}")

# Conversion en tableaux NumPy
data = np.array(data)
labels = np.array(labels)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Prétraitement terminé !")
print(f"Nombre d'images dans le jeu d'entraînement : {len(X_train)}")
print(f"Nombre d'images dans le jeu de test : {len(X_test)}")


Prétraitement des images de la classe : yeux_fermés
Prétraitement des images de la classe : yeux_ouverts
Prétraitement terminé !
Nombre d'images dans le jeu d'entraînement : 33434
Nombre d'images dans le jeu de test : 8359


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ---- Entraînement du modèle ----

# Définir le modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Sauvegarder le modèle
model.save('model_detection_fatigue.h5')

print("Entraînement terminé et modèle sauvegardé !")

# ---- Évaluation du modèle ----

# Charger le modèle sauvegardé
model = load_model('model_detection_fatigue.h5')

# Évaluer la performance du modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

# Prédictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Rapport de classification
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', xticklabels=['yeux_fermés', 'yeux_ouverts'], yticklabels=['yeux_fermés', 'yeux_ouverts'])
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.title('Matrice de confusion')
plt.show()

print("Évaluation du modèle terminée !")

ModuleNotFoundError: No module named 'tensorflow'